In [1]:
import glob
import json
import numpy as np

In [6]:
# old: "logs/2019-01-31-16-06-36" # black; "logs/2019-01-31-16-05-39" # grey
# folder = "logs/2019-01-31-19-09-50" # White
# folder = "logs/2019-01-31-19-10-28" # Grey
folder = "logs/2019-01-31-19-10-39" # Black
# folder = "logs/2019-02-01-11-56-49" # Problematic black
color = "black"
# color = "grey"
models = ["brp.v1.prism",
          "consensus.2.prism",
          "crowds.prism",
          "csma.2-2.prism",
          "firewire.true.prism",
          "haddad-monmege.pm",
          "ij.3.v1.prism",
          "ij.10.v1.prism",          
          "leader_sync.3-2.v1.prism",
          "pacman.nm",
          "philosophers-mdp.3.v1.prism",
          "pnueli-zuck.3.v1.prism",
          "rabin.3.prism",
          "wlan.0.prism",
          "zeroconf.prism",
          "cdmsn.prism",
          "cloud_5.prism",
          "mdsm.prism",
          "team-form-3.prism"
         ]

logs = {}

for model in models:
    l = glob.glob(folder+"/"+model+"*.log")
    s = glob.glob(folder+"/"+model+"*.stat")
    model_type = ""
    contents = open("models/"+model).read()
    if l and s:
        for t in ["smg\n", "mdp\n", "dtmc\n"]:
            if t in contents:
                #print(model+","+t.strip())
                model_type = [t.strip()]*len(l)
        logs[model] = list(zip(model_type, l, s))

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

for model in logs:
    for e in logs[model]:
        model_type, log_file, stat_file = e
        with open(log_file) as f:
            global_times = []
            precisions = []
            trials = []
            lower_bounds = []
            upper_bounds = []
            result = -1
            completed = -1
            prop, consts, color, _ = log_file.split("/")[2][len(model)+1:].split("-")
            for line in f:
                if line.startswith("JSON: "):
                    cur = json.loads(line[len("JSON: "):])
                    global_times.append(float(cur['GlobalTimerSecs']))
                    precisions.append(float(cur['Precision']))
                    trials.append(float(cur['Trials']))
                    lower_bounds.append(float(cur['Value']['Lower']))
                    upper_bounds.append(float(cur['Value']['Upper']))
                if line.startswith("Model checking completed in"):
                    completed = float(line.split(" ")[4])
                if line.startswith("Result"):
                    result = float(line.split(": ")[1])
            if len(global_times) == 0 and result < 0:
                continue
                #print("No result found for %s. Check log for errors." % model)
            else:
                plt.figure(figsize=(3.5,3.5))
                if len(global_times) == 1 and result >= 0:
                    plt.plot(global_times, precisions, 'b', label='Precision', linestyle='dashed', alpha=0.6, marker='o', markersize=3)
                    plt.plot(global_times, lower_bounds, 'r', label='Lower Bound', marker='o', markersize=3)
                    plt.plot(global_times, upper_bounds, 'g', label='Upper Bound', marker='o', markersize=3)
                    if completed >= 0:
                        plt.plot(completed, result, marker='*', markersize=10, color="orange", label="Result")
                    if color == "grey":
                        plt.xlim(-0.1, 240)
                else:
                    plt.plot(global_times, precisions, 'b', label='Precision', linestyle='dashed', alpha=0.6)
                    plt.plot(global_times, lower_bounds, 'r', label='Lower Bound')
                    plt.plot(global_times, upper_bounds, 'g', label='Upper Bound')
                    if completed >= 0:
                        plt.plot(completed, result, marker='*', markersize=10, color="orange", label="Result")
                    plt.xlim(-0.1, 3700)
                ax = plt.gca()
                ax.set_yticks(np.arange(0,1.1,0.1))
                plt.ylabel('Precision')
                plt.xlabel('Time (s)')
                #plt.title(log_file.split("/")[2][:-4])
                plt.legend()            
                #plt.grid(True)
                plt.ylim(-0.1,1.1)
                #plt.show()
                plt.savefig(folder+"/"+log_file.split("/")[2][:-4]+".png", bbox_inches='tight')
                plt.savefig(folder+"/"+log_file.split("/")[2][:-4]+".pdf", bbox_inches='tight')
                #print("Wrote graph to " + folder+"/"+log_file.split("/")[2][:-4]+".pdf")
                print("%s;%s;%s;%9f" % (model, consts, prop, precisions[-1]))
                plt.close()

brp.v1.prism;N=16,MAX=2;p1; 0.003020
brp.v1.prism;N=16,MAX=2;p1; 0.003480
brp.v1.prism;N=16,MAX=2;p1; 0.003067
consensus.2.prism;K=2;disagree; 0.007943
consensus.2.prism;K=2;disagree; 0.007987
consensus.2.prism;K=2;disagree; 0.007524
crowds.prism;TotalRuns=3,CrowdSize=5;positive; 0.007634
crowds.prism;TotalRuns=3,CrowdSize=5;positive; 0.007748
crowds.prism;TotalRuns=3,CrowdSize=5;positive; 0.007439
csma.2-2.prism;;some_before; 0.001057
csma.2-2.prism;;some_before; 0.001120
csma.2-2.prism;;some_before; 0.001093
firewire.true.prism;delay=3,deadline=200;deadline; 0.004085
firewire.true.prism;delay=3,deadline=200;deadline; 0.003775
firewire.true.prism;delay=3,deadline=200;deadline; 0.004162
haddad-monmege.pm;N=20,p=0.7;targt; 0.772056
haddad-monmege.pm;N=20,p=0.7;targt; 0.713713
haddad-monmege.pm;N=20,p=0.7;targt; 0.722295
ij.3.v1.prism;;stable; 0.000000
ij.3.v1.prism;;stable; 0.000000
ij.3.v1.prism;;stable; 0.000000
ij.10.v1.prism;;stable; 0.000000
ij.10.v1.prism;;stable; 0.000000
ij.10.v

In [ ]:
model = "consensus.2.prism"
glob.glob(folder+"/"+model+"*.log")